In [1]:
# https://chatbotslife.com/text-classification-using-algorithms-e4d50dcba45#.ig9im3fie

''' Hoy vamos a aproximarnos a cómo funcionan los chat-bots, x ejemplo Boti :)
Una de las partes más importantes es el 'CLASIFICADOR DE TEXTO', es decir, a
qué categoría asigna la frase que recibe: es un saludo, es un pedido???
Recordemos que la HIPÓTESIS FUERTE DE BAYES NAIVE ES QUE LAS CARACTERÍSTICAS
A CLASIFICAR SON INDEPENDIENTES ENTRE SÍ.
Se cumple ésto en las palabras de una frase? en general podemos decir que sí,
ya que NO estamos tratando de ENTENDER la frase, sino de asignarla a una
categoría. Es 'ingenuo' pero cierto.

Si yo digo: 'LA OVEJA SALTÓ LA CERCA ANOCHE', excepto los artículos femeninos
 LA,el resto de las palabras son independientes entre sí, ya que sólo están
conectadas por el SENTIDO de la frase y su CONTEXTO.
Si quisiera asignar la frase a una categoría, puedo decir que es un saludo? NO,
no hay ni HOLA, ni CÓMO ESTÁN? ni ninguna palabra que podamos relacionar con la
categoría 'saludos'.
Puedo decir que les estoy contando algo? Quizá la podemos clasificar como
'relatos de ovejas' si entrenamos adecuadamente al clasificador. SIEMPRE, LO
MÁS IMPORTANTE ES ENTRENAR APROPIADAMENTE AL CLASIFICADOR!!!!

Así que podemos suponer que estamos en la hipótesis de Bayes Ingenuo.

A estos efectos, en NLP (Procesamiento del Lenguaje Natural) a una oración
se la denomina 'Bolsa de palabras' (o 'bag of words').

PASOS A RECORRER

1. Importar librerías que necesitamos
    Usaremos NLTK para 2 procesos importantes:
        * TOKENIZAR: 'romper' la oración en cadenas de palabras separadas
                    ()
        * STEMMING: encuentra la 'raíz' de la palabra: 'corriendo' tiene como
                    raíz 'corr', como 'correré' o 'corrí' ('trocea' palabras)
                    Usaremos Lancaster Stemmer (buenas características para eng)

2. Proveer data de entrenamiento: definimos 'clases' para el entrenamiento

3. Organizar nuestros datos

4. Iterar: modelo + entrenamiento

5. Clasificar

'''


# usamos 'natural language toolkit'  https://www.nltk.org/
import nltk
nltk.download('punkt')  # esta línea depende de cómo tengo mi OS, y puede cambiar
from nltk.stem.lancaster import LancasterStemmer
# 'troceador' de palabras
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# 3 clases de datos para entrenamiento
training_data = []
training_data.append({"clase":"saludo", "oración":"how are you?"})
training_data.append({"clase":"saludo", "oración":"how is your day?"})
training_data.append({"clase":"saludo", "oración":"good day"})
training_data.append({"clase":"saludo", "oración":"how is it going today?"})

training_data.append({"clase":"despedida", "oración":"have a nice day"})
training_data.append({"clase":"despedida", "oración":"see you later"})
training_data.append({"clase":"despedida", "oración":"have a good night"})
training_data.append({"clase":"despedida", "oración":"talk to you soon"})

training_data.append({"clase":"sandwich", "oración":"make me a sandwich"})
training_data.append({"clase":"sandwich", "oración":"can you make a sandwich?"})
training_data.append({"clase":"sandwich", "oración":"having a sandwich today?"})
training_data.append({"clase":"sandwich", "oración":"what's for lunch?"})

print (f"{len(training_data)} oraciones para entrenamiento")

12 oraciones para entrenamiento


In [3]:
# se queda con palabras trozadas únicas en el cjto de entrenamiento
# CORPUS es una palabra de NLP, se refiere al conjunto total de palabras
corpus_palabras = {}  # diccionario que cuenta cuántas palabras trozadas hay
clase_palabras = {}  # diccionario donde se agregan las palabras según la clase
# convierte la lista en conjunto (set de ítems únicos) y luego a lista otra vez
clases = list(set([a['clase'] for a in training_data]))
for c in clases:
    # prepara una lista de palabras dentro de cada clase
    clase_palabras[c] = []

# itera dentro de cada oración en el cjto de entrenamiento
for data in training_data:
    # tokeniza cada oración en palabras como cadenas
    for palabra in nltk.word_tokenize(data['oración']):
        # ignora algunas cosas
        if palabra not in ["?", "'s"]:
            # trocea y pone en minúsculas cada palabra
            stemmed_palabra = stemmer.stem(palabra.lower())
            # vimos esta palabra antes?
            if stemmed_palabra not in corpus_palabras:
                corpus_palabras[stemmed_palabra] = 1
            else:
                corpus_palabras[stemmed_palabra] += 1

            # agrega la palabra a nuestras palabras en la lista de clases
            clase_palabras[data['clase']].extend([stemmed_palabra])

# ahora sabemos cuántas palabras trozadas hay y cuántas veces aparecen
print (f"Corpus de palabras y cantidad: {corpus_palabras} /n" )
# también sabemos cuántas clases de palabras hay y cuáles son ellas
print (f"Clases de palabras: {clase_palabras}")

Corpus de palabras y cantidad: {'how': 3, 'ar': 1, 'you': 4, 'is': 2, 'yo': 1, 'day': 3, 'good': 2, 'it': 1, 'going': 1, 'today': 2, 'hav': 3, 'a': 5, 'nic': 1, 'see': 1, 'lat': 1, 'night': 1, 'talk': 1, 'to': 1, 'soon': 1, 'mak': 2, 'me': 1, 'sandwich': 3, 'can': 1, 'what': 1, 'for': 1, 'lunch': 1} /n
Clases de palabras: {'sandwich': ['mak', 'me', 'a', 'sandwich', 'can', 'you', 'mak', 'a', 'sandwich', 'hav', 'a', 'sandwich', 'today', 'what', 'for', 'lunch'], 'saludo': ['how', 'ar', 'you', 'how', 'is', 'yo', 'day', 'good', 'day', 'how', 'is', 'it', 'going', 'today'], 'despedida': ['hav', 'a', 'nic', 'day', 'see', 'you', 'lat', 'hav', 'a', 'good', 'night', 'talk', 'to', 'you', 'soon']}


In [4]:
# Ahora hay que 'codear'el algoritmo

# Primera aproximación: suponemos que todas las palabras de la oracion tienen
# el mismo 'peso' en la clasificación (no importa si son más o menos comunes)

# calcula el score para una clase dada
def calcula_score_clase(oracion, clase, show_details=True):
    score = 0
    # tokeniza cada palabra de la nueva oración
    for palabra in nltk.word_tokenize(oracion):
        # mira si el stem de la palabra está en alguna de nuestras clases
        if stemmer.stem(palabra.lower()) in clase_palabras[clase]:
            # asigna a cada palabra el mismo peso
            score += 1

            if show_details:
                print (f"   encontré: {stemmer.stem(palabra.lower() )}")
    return score

# la oración fue tokenizada y las palabras fueron trozadas y pasadas a minúsculas
# como los datos de entrenamiento

In [5]:
# podemos calcular el score para una oración nueva
oracion = "good day for us to have lunch?"

for c in clase_palabras.keys():
    print (f"Clase: {c}  Score: {calcula_score_clase(oracion, c)}")



   encontré: for
   encontré: hav
   encontré: lunch
Clase: sandwich  Score: 3
   encontré: good
   encontré: day
Clase: saludo  Score: 2
   encontré: good
   encontré: day
   encontré: to
   encontré: hav
Clase: despedida  Score: 4


In [6]:
# 2da Aproximación: hay palabras que aparecen con más frecuencia en las
# oraciones, o sea, son más 'comunes'. Podemos pensar que por aparecer más
# veces podrían 'confundir' al clasificador para asignar la clase a la oración

# calcula el score para una dada clase tomando en cuenta qué tan 'comunes' son
# las palabras, es decir, cuántas veces aparecen
def calcula_score_clase_comun(oracion, clase, show_details=True):
    score = 0
    # tokeniza la oración en palabras separadas
    for palabra in nltk.word_tokenize(oracion):
        # mira si la raíz de la palabra está en alguna de nuestra clases
        if stemmer.stem(palabra.lower()) in clase_palabras[clase]:
            # trata a cada palabra con su peso relativo
            score += (1 / corpus_palabras[stemmer.stem(palabra.lower())])

            if show_details:
                print (f"   encontré: {stemmer.stem(palabra.lower())} {1 / corpus_palabras[stemmer.stem(palabra.lower())]}")

    return score


In [7]:
# ahora podemos ver qué pasa con la oración anterior

for c in clase_palabras.keys():
    print (f"Clase: {c}  Score: {calcula_score_clase_comun(oracion, c)}")


   encontré: for 1.0
   encontré: hav 0.3333333333333333
   encontré: lunch 1.0
Clase: sandwich  Score: 2.333333333333333
   encontré: good 0.5
   encontré: day 0.3333333333333333
Clase: saludo  Score: 0.8333333333333333
   encontré: good 0.5
   encontré: day 0.3333333333333333
   encontré: to 1.0
   encontré: hav 0.3333333333333333
Clase: despedida  Score: 2.1666666666666665


In [8]:
# 3ra Aproximación (y última): buscar la clase con más score y asignarle la oración

def clasificar(oracion):
    clase_mas_alta = None
    score_mas_alto = 0
    # iteramos entre las clases
    for c in clase_palabras.keys():
        # calcula score de la oracion para cada clase
        score = calcula_score_clase_comun(oracion, c, show_details=False)
        # mira cuál es el score más alto
        if score > score_mas_alto:
            clase_mas_alta = c
            score_mas_alto = score

    return clase_mas_alta, score_mas_alto


In [9]:
clasificar(oracion)

('sandwich', 2.333333333333333)

In [10]:
clasificar("make me some lunch?")

('sandwich', 2.5)

In [11]:
clasificar("sudo make me a sandwich")

('sandwich', 2.033333333333333)

In [12]:
clasificar("how are you doing today?")

('saludo', 2.083333333333333)

In [13]:
clasificar("talk to you tomorrow")

('despedida', 2.25)

In [14]:
clasificar("who are you?")

('saludo', 1.25)

In [15]:

clasificar("am I crazy?")

# Bayes Ingenuo? puede ser pero FUNCIONA!!!!

(None, 0)

In [16]:
clasificar('I have a cat')

('sandwich', 0.5333333333333333)